In [46]:
import weave
import pandas as pd
from ydata_profiling import ProfileReport

from core import CasualModelingAgentMachine, LLMSession
from specifications import SPECIFICATION_V0
from data_config import CENSUS_DATASET_METADATA, OLD_CENSUS_DATASET_METADATA
from utils import sample_dataset, metadata_to_pandera_schema

RANDOM_STATE = 42

weave.init("data_gen_agent")

def produce_dataset(num_samples):
    try:

        llm_sess = LLMSession(
            specification=SPECIFICATION_V0,
            metadata=CENSUS_DATASET_METADATA)

        _ = CasualModelingAgentMachine(llm_sess)

        pandera_schema = metadata_to_pandera_schema(CENSUS_DATASET_METADATA["schema"])

        df = sample_dataset(llm_sess.context["model"], num_samples, pandera_schema)
        
        code = llm_sess.context["code"]

        error = None

    except Exception as e:
        df, code, error = None, None, e

    return df, code, error


def produce_mixture_dataset(num_samples, num_datasets, random_state=RANDOM_STATE):
    dfs = []
    codes = []
    errors = []

    while len(dfs) < num_datasets:
        print(len(dfs))

        df, code, error = produce_dataset(num_samples)
        if error is None:
            dfs.append(df)
            codes.append(code)
        else:
            errors.append(error)    
            print(error)

    mixture_df = (pd.concat(dfs)
                  .sample(num_samples, replace=False, random_state=random_state)
                  .reset_index(drop=True))

    return mixture_df, (dfs, codes, errors)

In [ ]:
mixture_df, details = produce_mixture_dataset(23_006, 5)

0
🍩 https://wandb.ai/shlomihod/data_gen_agent/r/call/01932a84-3a66-7342-9183-9520af9a5e27
🍩 https://wandb.ai/shlomihod/data_gen_agent/r/call/01932a84-4612-7ed1-b364-8bd00e2efce3
🍩 https://wandb.ai/shlomihod/data_gen_agent/r/call/01932a84-8734-7472-9b0b-72fbfbc6adb0
🍩 https://wandb.ai/shlomihod/data_gen_agent/r/call/01932a84-a31f-77b1-9dff-a5939b4b28de
🍩 https://wandb.ai/shlomihod/data_gen_agent/r/call/01932a84-c1d7-7b52-901a-e3ac45005d23
🍩 https://wandb.ai/shlomihod/data_gen_agent/r/call/01932a84-db53-7983-9116-784f797edf88
🍩 https://wandb.ai/shlomihod/data_gen_agent/r/call/01932a84-db54-75b3-a6d5-162ef32bcde5
🍩 https://wandb.ai/shlomihod/data_gen_agent/r/call/01932a85-0316-7d33-9636-8a6956940d75
🍩 https://wandb.ai/shlomihod/data_gen_agent/r/call/01932a85-303a-7260-a2b0-9ed3b2a4f180
🍩 https://wandb.ai/shlomihod/data_gen_agent/r/call/01932a85-7daa-7533-ac11-87a2b45bf081
🍩 https://wandb.ai/shlomihod/data_gen_agent/r/call/01932a85-cdf1-7650-9d88-a4c44a90170b
🍩 https://wandb.ai/shlomihod/d

In [ ]:
# mixture_df.to_csv("acs-mix.csv")

In [ ]:
# import sys
# import os
# os.chdir("..")
# from ydnpd import load_dataset
# national_df = load_dataset("acs/national")[0]

In [45]:
(ProfileReport(national_df, title="National")
 .compare(
     ProfileReport(mixture_df, title="Agent")
 )
)

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]